In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('GPT_API_KEY')
if api_key is None:
  raise

headers = {
  "Content-Type": "application/json",
  "X-Merck-APIKey": api_key
}

In [ ]:
import requests
import json

GPT_URL = "https://iapi-test.merck.com/gpt/v2/gpt-35-turbo-0613/chat/completions?api-version=2023-08-01-preview"
rag_call_convo = [
  {"role": "system", "content": """You are a documentation generation assistant"""},
]
rag_call_question = "What do you like to know today?"
url = ""

while url == "":
  user_msg = input(rag_call_question)
  rag_call_convo.append({"role": "user", "content": user_msg})
  payload = {
    "messages":rag_call_convo,
    "functions": [{
      "name": "retrieve_doc_with_url",
      "description": "Retrieve document with a web url",
      "parameters": {
        "type": "object",
        "properties": {
          "url": {
            "type": "string",
            "description": "A web url"
          }
        },
        "required": ["url"]
      }
    }]
  }
  res = requests.post(GPT_URL, json=payload, headers=headers)
  output = res.json()
  if "status" in output and output["status"] != 200:
    print(output)
    raise
  choice = output['choices'][0]
  reason = choice['finish_reason']
  if reason == "function_call":
    arg_text = choice['message']['function_call']['arguments']
    args = json.loads(arg_text)
    url = args['url']
    print(url)
  else:
    rag_call_question = choice['message']['content']
    print(rag_call_question)
    rag_call_convo.append({"role": "assistant", "content": rag_call_question})
